In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
%matplotlib inline

In [2]:
df_p = [pd.read_csv(file,index_col="ts") for file in glob.glob('./Raw Data/Chiller1_power/*.csv')]
df_t = [pd.read_csv(file,index_col="ts") for file in glob.glob('./Raw Data/Chiller1_temp/*.csv')]
df_e = [pd.read_csv(file,index_col="ts") for file in glob.glob('./Raw Data/Chiller1_evaflow/*.csv')]
df_c = [pd.read_csv(file,index_col="ts") for file in glob.glob('./Raw Data/Chiller1_conflow/*.csv')]

In [3]:
tab_p=pd.concat(df_p).sort_index()
tab_p=tab_p[["ch1Watt","ch2Watt","ch3Watt","totalPositiveWattHour"]]

tab_t=pd.concat(df_t).sort_index()
tab_t=tab_t[["value1","value2","value3","value4"]]
tab_t.columns=["temp1","temp2","temp3","temp4"]

tab_e=pd.concat(df_e).sort_index()
tab_e=tab_e[["flowRate","flowSpeed","totalFlowRate"]]
tab_e.columns=["e.flowRate","e.flowSpeed","e.totalFlowRate"]

tab_c=pd.concat(df_c).sort_index()
tab_c=tab_c[["flowRate","flowSpeed","totalFlowRate"]]
tab_c.columns=["c.flowRate","c.flowSpeed","c.totalFlowRate"]

In [4]:
tab_e[tab_e['e.totalFlowRate']<0]

,e.flowRate,e.flowSpeed,e.totalFlowRate
ts,,,
2017-07-25 00:09:03.427,220.466461,3.284412,-2130685116
2017-09-27 11:05:17.788,0.000000,0.000000,-1795122789


In [5]:
tab_e.loc['2017-07-25 00:09:03.427','e.totalFlowRate']=217369

In [6]:
tab_e.loc['2017-09-27 11:05:17.788','e.totalFlowRate']=300511

In [7]:
tab_e.describe()

,e.flowRate,e.flowSpeed,e.totalFlowRate
count,293430.000000,293430.000000,293430.000000
mean,40.050261,0.596651,244886.809065
std,82.215534,1.224811,61839.150211
min,0.000000,0.000000,124807.000000
25%,0.000000,0.000000,179300.000000
50%,0.000000,0.000000,257998.000000
75%,14.230552,0.212000,303488.750000
max,233.577560,3.479735,318853.000000


In [8]:
tab_c[tab_c['c.totalFlowRate']<190000]

,c.flowRate,c.flowSpeed,c.totalFlowRate
ts,,,
2017-05-10 14:39:24.901,1.409711e-38,2.258878e+15,2304
2017-06-18 21:10:17.948,0.000000e+00,0.000000e+00,-1717895015


In [9]:
tab_c.loc['2017-05-10 14:39:24.901','c.totalFlowRate']=218047

In [10]:
tab_c.loc['2017-06-18 21:10:17.948','c.totalFlowRate']=239261

In [11]:
tab_c.describe()

,c.flowRate,c.flowSpeed,c.totalFlowRate
count,293416.000000,2.934160e+05,293416.000000
mean,39.229280,7.698551e+09,311754.166719
std,85.851586,4.170142e+12,61696.059725
min,0.000000,0.000000e+00,190527.000000
25%,0.000000,0.000000e+00,243881.000000
50%,0.000000,0.000000e+00,327736.000000
75%,0.000000,0.000000e+00,371635.000000
max,237.545517,2.258878e+15,377397.000000


In [12]:
tables=[tab_p,tab_t,tab_e,tab_c]

In [13]:
for i in tables:
    i.index=pd.DatetimeIndex(i.index)
    i.index=i.index.map(lambda x:x.replace(minute=0,second=0,microsecond=0))
print(tab_p.shape,tab_t.shape,tab_e.shape,tab_c.shape)

(295954, 4) (301943, 4) (293430, 3) (293416, 3)


In [14]:
result_p = tab_p.groupby('ts').mean()
result_p['totalPositiveWattHour'] = tab_p['totalPositiveWattHour'].groupby('ts').min()-tab_p['totalPositiveWattHour'].min()

In [15]:
result_t = tab_t.groupby('ts').mean()

In [16]:
result_e = tab_e.groupby('ts').mean()
result_e['e.totalFlowRate'] = tab_e['e.totalFlowRate'].groupby('ts').min()-tab_e['e.totalFlowRate'].min()

In [17]:
result_c = tab_c.groupby('ts').mean()
result_c['c.totalFlowRate'] = tab_c['c.totalFlowRate'].groupby('ts').min()-tab_c['c.totalFlowRate'].min()

In [18]:
all_days = pd.date_range(result_p.index.min(), result_p.index.max(), freq='H')
results=[result_p,result_t,result_e,result_c]
for j in results:
    

In [19]:
c1_final=result_p.merge(result_t,left_index=True, right_index=True).merge(result_e,left_index=True, right_index=True).merge(result_c,left_index=True, right_index=True)

In [20]:
c1_final

,ch1Watt,ch2Watt,ch3Watt,totalPositiveWattHour,temp1,temp2,temp3,temp4,e.flowRate,e.flowSpeed,e.totalFlowRate,c.flowRate,c.flowSpeed,c.totalFlowRate
ts,,,,,,,,,,,,,,
2017-05-01 00:00:00,54214.357143,50918.589286,52073.500000,0.0,12.421112,16.275228,29.816259,26.318716,207.265389,3.087748,0,221.046209,1.902006,0
2017-05-01 01:00:00,30305.392857,26247.803571,28257.714286,156600.0,8.701223,11.235682,28.610714,26.630606,225.610802,3.361050,210,229.764071,1.977019,223
2017-05-01 02:00:00,17133.333333,13040.438596,14700.964912,240400.0,9.398168,11.133499,24.589259,23.434271,225.662504,3.361820,436,227.765573,1.959823,453
2017-05-01 03:00:00,16940.946429,12807.750000,14721.910714,285500.0,9.646420,11.378396,24.020174,22.869687,225.772536,3.363459,662,226.770818,1.951264,681
2017-05-01 04:00:00,17188.875000,12808.803571,14656.392857,328800.0,9.635645,11.379343,24.020132,22.866923,225.691767,3.362256,887,226.592686,1.949731,907
2017-05-01 05:00:00,16945.678571,12466.750000,14402.232143,372900.0,9.629237,11.337331,24.011765,22.855352,225.746851,3.363077,1113,226.745203,1.951043,1134
2017-05-01 06:00:00,16603.649123,12479.912281,14180.140351,416800.0,9.649531,11.341425,24.088670,22.923890,225.746214,3.363067,1339,227.088373,1.953996,1361
2017-05-01 07:00:00,16495.392857,12647.303571,13958.267857,460200.0,9.770068,11.498785,24.638595,23.468194,225.557999,3.360263,1565,228.571231,1.966755,1588
2017-05-01 08:00:00,19042.535714,15032.482143,16486.642857,504100.0,10.199966,12.068100,25.710246,24.391658,226.132657,3.368824,1790,226.783416,1.951372,1816


In [21]:
c1_final.describe()

,ch1Watt,ch2Watt,ch3Watt,totalPositiveWattHour,temp1,temp2,temp3,temp4,e.flowRate,e.flowSpeed,e.totalFlowRate,c.flowRate,c.flowSpeed,c.totalFlowRate
count,5018.000000,5018.000000,5018.000000,5.018000e+03,5018.000000,5018.000000,5018.000000,5018.000000,5018.000000,5018.000000,5018.000000,5018.000000,5.018000e+03,5018.000000
mean,8245.755849,7296.812003,7919.827830,7.612493e+07,13.402915,17.310255,28.746598,28.658531,38.027254,0.566513,120760.200478,36.576841,1.875646e+10,122135.109805
std,18919.943120,17631.489272,18209.998478,4.004014e+07,3.351532,6.229809,3.506368,3.172445,78.007823,1.162126,62947.545100,81.037974,1.328667e+12,62739.749734
min,321.766667,0.000000,335.540984,0.000000e+00,5.533371,6.296005,19.380773,19.499690,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,341.500000,0.000000,351.082309,3.169428e+07,10.900921,11.838781,26.718790,26.565333,0.000000,0.000000,51177.000000,0.000000,0.000000e+00,52135.000000
50%,345.533333,0.000000,356.721767,8.971010e+07,12.772077,14.997228,29.037592,29.091806,0.000000,0.000000,144073.000000,0.000000,0.000000e+00,148364.000000
75%,350.900000,0.000000,366.896702,1.142073e+08,16.405850,23.046990,30.996794,31.024216,14.615481,0.217735,178985.500000,0.000000,0.000000e+00,181108.000000
max,75575.066667,71037.800000,72906.050000,1.189562e+08,24.037044,35.801574,38.811761,37.741675,230.199586,3.429412,194046.000000,235.573485,9.411991e+13,186870.000000


In [22]:
c1_final.info()
#May-Nov has (31 + 30 + 31 + 31 + 30 + 31 + 30)(24) = 5136 hours in total.  There should be 5136 entries

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5018 entries, 2017-05-01 00:00:00 to 2017-11-30 23:00:00
Data columns (total 14 columns):
ch1Watt                  5018 non-null float64
ch2Watt                  5018 non-null float64
ch3Watt                  5018 non-null float64
totalPositiveWattHour    5018 non-null float64
temp1                    5018 non-null float64
temp2                    5018 non-null float64
temp3                    5018 non-null float64
temp4                    5018 non-null float64
e.flowRate               5018 non-null float64
e.flowSpeed              5018 non-null float64
e.totalFlowRate          5018 non-null int64
c.flowRate               5018 non-null float64
c.flowSpeed              5018 non-null float64
c.totalFlowRate          5018 non-null int64
dtypes: float64(12), int64(2)
memory usage: 748.0 KB


In [24]:
c1_final.to_csv('Chiller1 Data (May-Nov)(Hourly).csv')